In [ ]:
%load_ext autoreload
%autoreload 2

# MusicGen-RepEng
Welcome to MusicGen-RepEng's demo jupyter notebook. Here you will find a series of self-contained examples of how to use MusicGen with Representation Engineering.

First, we start by initializing MusicGen, you can choose a model from the following selection:
1. `facebook/musicgen-small` - 300M transformer decoder.
2. `facebook/musicgen-medium` - 1.5B transformer decoder.
3. `facebook/musicgen-melody` - 1.5B transformer decoder also supporting melody conditioning.
4. `facebook/musicgen-large` - 3.3B transformer decoder.

We will use the `facebook/musicgen-small` variant for the purpose of this demonstration.

In [ ]:
from audiocraft.models import MusicGen
from audiocraft.models import MultiBandDiffusion
from audiocraft.utils.notebook import display_audio

USE_DIFFUSION_DECODER = False
# Using small model, better results would be obtained with `medium` or `large`.
model = MusicGen.get_pretrained('facebook/musicgen-large', device="cuda")
if USE_DIFFUSION_DECODER:
    mbd = MultiBandDiffusion.get_mbd_musicgen()

Next, let us configure the generation parameters. Specifically, you can control the following:
* `use_sampling` (bool, optional): use sampling if True, else do argmax decoding. Defaults to True.
* `top_k` (int, optional): top_k used for sampling. Defaults to 250.
* `top_p` (float, optional): top_p used for sampling, when set to 0 top_k is used. Defaults to 0.0.
* `temperature` (float, optional): softmax temperature parameter. Defaults to 1.0.
* `duration` (float, optional): duration of the generated waveform. Defaults to 30.0.
* `cfg_coef` (float, optional): coefficient used for classifier free guidance. Defaults to 3.0.

When left unchanged, MusicGen will revert to its default parameters.

In [ ]:
model.set_generation_params(
    use_sampling=True,
    top_k=250,
    duration=3
)

Next, we can go ahead and start generating music using one of the following modes:
* Unconditional samples using `model.generate_unconditional`
* Music continuation using `model.generate_continuation`
* Text-conditional samples using `model.generate`
* Melody-conditional samples using `model.generate_with_chroma`

In [ ]:
n = 64
res = model.generate(descriptions=[None]*n, progress=True)
display_audio(res, 32000)

In [ ]:
import torchaudio

ditto, sr = torchaudio.load('/home/sake/ditto_vocalless.mp3')

In [ ]:
n = 64
res = model.generate_continuation(descriptions=[None]*n, prompt=ditto[...,30*sr:int(31.5*sr)].repeat(n,1,1), prompt_sample_rate=sr, progress=True)
display_audio(res, 32000)

In [ ]:
n = 128
res = model.get_hidden_states_no_continuation(descriptions=[None]*n, progress=True)

In [ ]:
len(res), res[-1].shape

In [ ]:
n = 4
res = model.get_hidden_states_no_continuation(descriptions=[None]*n, progress=True)

### Get Distance Mean/Var

In [ ]:
import torch
from tqdm import tqdm
from pathlib import Path
from natsort import natsorted
import matplotlib.pyplot as plt

def get_distance(target_path, opposite_path):
    target_representations = []
    for p in tqdm(natsorted(Path(target_path).rglob('*.pt'), key=str)):
        loaded = torch.load(str(p), map_location=torch.device('cpu')).to(torch.float32)
        target_representations.append(loaded)

    target_representations = torch.cat(target_representations, dim=0)

    opposite_representations = []
    for p in tqdm(natsorted(Path(opposite_path).rglob('*.pt'), key=str)):
        loaded = torch.load(str(p), map_location=torch.device('cpu')).to(torch.float32)
        opposite_representations.append(loaded)

    opposite_representations = torch.cat(opposite_representations, dim=0)

    dist = (target_representations - opposite_representations)
    dist = dist.permute(1,0,2)
    dist = dist.pow(2).sum(-1).sqrt()

    return dist

In [ ]:
target_path = "/home/sake/MusicGenRepEng_2.5s_ditto_hiddens_batch128_meanssss"
opposite_path = "/home/sake/MusicGenRepEng_125tokens_uncond_hiddens_batch128_meanssss"
dist = get_distance(target_path, opposite_path)

In [ ]:
dist

In [ ]:
plt.plot(dist.mean(-1), label='mean')
plt.plot(dist.var(-1), label='var')
plt.legend()

In [ ]:
ditto_dist_mean = dist.mean(-1)

In [ ]:
ditto_dist_mean = ditto_dist_mean/ditto_dist_mean.max()

In [ ]:
ditto_dist_mean

### PCA Plot

In [ ]:
import torch
from tqdm import tqdm
from pathlib import Path
from natsort import natsorted
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

def plot_pca(target_path, opposite_path):
    target_representations = []
    for p in tqdm(natsorted(Path(target_path).rglob('*.pt'), key=str)):
        loaded = torch.load(str(p), map_location=torch.device('cpu'))
        target_representations.append(loaded)

    target_representations = torch.cat(target_representations, dim=0)

    opposite_representations = []
    for p in tqdm(natsorted(Path(opposite_path).rglob('*.pt'), key=str)):
        loaded = torch.load(str(p), map_location=torch.device('cpu'))
        opposite_representations.append(loaded)

    opposite_representations = torch.cat(opposite_representations, dim=0)

    representations = torch.cat([target_representations, opposite_representations], dim=0)
    representations = representations.permute(1,0,2)

    for layer, representation in enumerate(representations):
        scaler = StandardScaler()
        scaler.fit(representation)
        representation_scaled = scaler.transform(representation)
        pca = PCA(n_components=2).fit(representation_scaled)
        representation_pca = pca.transform(representation_scaled)
        
        target_representation_pca = representation_pca[:len(target_representations)]
        opposite_representation_pca = representation_pca[len(target_representations):]

        plt.scatter(target_representation_pca[:,0], target_representation_pca[:,1], color = 'red', alpha = 0.3, s=1, label = 'ditto')
        plt.scatter(opposite_representation_pca[:,0], opposite_representation_pca[:,1], color = 'green', alpha = 0.3, s=1, label = 'oppposite')

        plt.xlabel('component 0')
        plt.ylabel('component 1')
        plt.title(f"Layer {layer}")
        plt.legend()
        plt.show()

def plot_pca_over_opposite(target_path, opposite_path):
    target_representations = []
    for p in tqdm(Path(target_path).rglob('*.pt')):
        loaded = torch.load(str(p), map_location=torch.device('cpu'))
        target_representations.append(loaded)

    target_representations = torch.cat(target_representations, dim=0)
    target_representations = target_representations.permute(1,0,2)

    opposite_representations = []
    for p in tqdm(Path(opposite_path).rglob('*.pt')):
        loaded = torch.load(str(p), map_location=torch.device('cpu'))
        opposite_representations.append(loaded)

    opposite_representations = torch.cat(opposite_representations, dim=0)
    opposite_representations = opposite_representations.permute(1,0,2)

    for layer, zipped in enumerate(zip(target_representations, opposite_representations)):
        target_representation, opposite_representation = zipped
        scaler = StandardScaler()
        scaler.fit(opposite_representation)
        target_representation_scaled = scaler.transform(target_representation)
        opposite_representation_scaled = scaler.transform(opposite_representation)
        pca = PCA(n_components=2).fit(opposite_representation_scaled)
        opposite_representation_pca = pca.transform(opposite_representation_scaled)
        target_representation_pca = pca.transform(target_representation_scaled)

        plt.scatter(target_representation_pca[:,0], target_representation_pca[:,1], color = 'red', alpha = 0.3, s=1, label = 'ditto')
        plt.scatter(opposite_representation_pca[:,0], opposite_representation_pca[:,1], color = 'green', alpha = 0.3, s=1, label = 'oppposite')

        plt.xlabel('component 0')
        plt.ylabel('component 1')
        plt.title(f"Layer {layer}")
        plt.legend()
        plt.show()

In [ ]:
target_path = "/home/sake/MusicGenRepEng_2.5s_ditto_hiddens_batch128_meanssss"
opposite_path = "/home/sake/MusicGenRepEng_125tokens_uncond_hiddens_batch128_meanssss"
plot_pca(target_path, opposite_path)
# plot_pca_over_opposite(target_path, opposite_path)

In [ ]:
target_path = "/home/sake/MusicGenRepEng_2.5s_ditto_hiddens_batch128_meanssss"
opposite_path = "/home/sake/MusicGenRepEng_125tokens_uncond_hiddens_batch128_meanssss"
# plot_pca(target_path, opposite_path)
plot_pca_over_opposite(target_path, opposite_path)

### MISCS

In [ ]:
import torchaudio

music, sr = torchaudio.load('/home/sake/MusicGenRepEng_Dataset/Rock/Alternative Rock/Nirvana - Smells Like Teen Spirit.mp3')

In [ ]:
import torch

In [ ]:
(music[:,:50].repeat(2,1,1) == torch.stack([music[:,:50], music[:,:50]], dim=0)).all()

In [ ]:
from audiocraft.utils.notebook import display_audio

model.set_generation_params(
    use_sampling=True,
    top_k=250,
    duration=8,
    two_step_cfg=False
)

# Here we use a synthetic signal to prompt both the tonality and the BPM
# of the generated audio.
res = model.generate_continuation(
    music[:,:int(sr*0.02)].repeat(2,1,1),
    sr, ['rock, energetic', 
            'rock, sleepy'], 
    progress=True)
display_audio(res, 32000)

In [ ]:
res.shape

### Get Text Condition Representations(Hidden States)

In [ ]:
import torchaudio

music, sr = torchaudio.load('/home/sake/MusicGenRepEng_Dataset/Rock/Alternative Rock/Nirvana - Smells Like Teen Spirit.mp3')

In [ ]:
model.set_generation_params(
    use_sampling=True,
    top_k=250,
    duration=0.02,
    two_step_cfg=True
)

In [ ]:
hidden_states = model.get_hidden_states(music[:,:int(sr*0.02)].repeat(2,1,1), sr, ["fast tempo", "slow tempo"])

In [ ]:
hidden_states[1][1].shape

In [ ]:
hidden_states[1][1].shape

In [ ]:
attributes, _ = model._prepare_tokens_and_attributes(["techno, fast beats, happy, hard, joyful, tribal"], None)
attributes, _

In [ ]:
embeddings = model.lm.condition_provider(model.lm.condition_provider.tokenize(attributes))
embeddings

In [ ]:
embeddings['description']

In [ ]:
embeddings['description'][0].shape

In [ ]:
attributes = model.get_hidden_states_text_condition(["techno, fast beats"])

In [ ]:
attributes[0]['text']

In [ ]:
model.lm.condition_provider.conditioners

In [ ]:
model.lm.condition_provider

In [ ]:
model.lm.condition_provider(model.lm.condition_provider.tokenize(attributes))

In [ ]:
model.lm.condition_provider.conditioners.description(attributes[0]['text']['description'])

In [ ]:
model.lm.condition_provider(attributes[0]['text'])

### Get Representations(Hidden States)

In [ ]:
import torchaudio

music, sr = torchaudio.load('/home/sake/MusicGenRepEng_Dataset/Rock/Alternative Rock/Nirvana - Smells Like Teen Spirit.mp3')

In [ ]:
music.shape

In [ ]:
i = 0

In [ ]:
input_music = music[:, 20*i*sr:20*(i+1)*sr]

In [ ]:
rep = model.get_hidden_states(
    input_music, 
    sr, None, 
    progress=True)

In [ ]:
reps = []
for i in range(10):
    input_music = music[:, 20*i*sr:20*(i+1)*sr]
    rep = model.get_hidden_states(
        input_music,
        sr, None,
        progress=True)
    reps.append(rep)

In [ ]:
len(reps)

In [ ]:
import torch

In [ ]:
rep_vec = torch.stack(rep, dim=1)[:,500:1000]

In [ ]:
rep_vec.shape

In [ ]:
torch.save(reps, "/home/sake/Nirvana - Smells Like Teen Spirit_MusicGenRepEng_Dataset_hidden_states_every20s_10t.pt")

In [ ]:
from pathlib import Path
from tqdm import tqdm 

for path in tqdm(Path('/home/sake/MusicGenRepEng_Dataset_separated').rglob('*.mp3')):
    print("Representing: ", path)
    out_path = str(path).replace('MusicGenRepEng_Dataset', 'MusicGenRepEng_Dataset_hidden_states_30-60_mid10')
    Path(out_path).parent.mkdir(parents=True, exist_ok=True)
    music, sr = torchaudio.load(str(path))
    input_music = music[:, 30*sr:50*sr]
    rep = model.read_representations(
        input_music, 
        sr, None, 
        progress=True)
    rep_vec = torch.stack(rep, dim=1)[:,500:1000].mean(1)
    torch.save(rep_vec, path.with_suffix('.pt'))

### Get Control Vector - Ditto

In [ ]:
import torch
import numpy as np
from tqdm import tqdm
from pathlib import Path
from natsort import natsorted
from sklearn.decomposition import PCA

def project_onto_direction(H, direction):
    """Project matrix H (n, d_1) onto direction vector (d_2,)"""
    mag = np.linalg.norm(direction)
    assert not np.isinf(mag)
    return (H @ direction) / mag

def get_directions(target_path, opposite_path):
    target_representations = []
    for p in tqdm(natsorted(Path(target_path).rglob('*.pt'), key=str)):
        loaded = torch.load(str(p), map_location=torch.device('cpu'))
        target_representations.append(loaded)

    target_representations = torch.cat(target_representations, dim=0)

    opposite_representations = []
    for p in tqdm(natsorted(Path(opposite_path).rglob('*.pt'), key=str)):
        loaded = torch.load(str(p), map_location=torch.device('cpu'))
        opposite_representations.append(loaded)

    opposite_representations = torch.cat(opposite_representations, dim=0)

    representation_pairs = torch.empty((target_representations.shape[0] * 2, target_representations.shape[1], target_representations.shape[2]))
    representation_pairs[0::2] = target_representations
    representation_pairs[1::2] = opposite_representations
    representation_pairs = representation_pairs.permute(1,0,2)

    relative_layer_hiddens = {}
    for layer, pair in enumerate(representation_pairs):
        relative_layer_hiddens[layer] = (
            pair[::2] - pair[1::2]
        )

    directions = {}
    for layer in range(len(relative_layer_hiddens)):
        # assert representation_pairs[layer].shape[0] == 110 * 2

        # fit layer directions
        train = np.vstack(
            relative_layer_hiddens[layer].to("cpu").numpy()
            - relative_layer_hiddens[layer].to("cpu").numpy().mean(axis=0, keepdims=True)
        )
        pca_model = PCA(n_components=1, whiten=False).fit(train)
        # shape (n_features,)
        directions[layer] = pca_model.components_.astype(np.float32).squeeze(axis=0)

        # calculate sign
        projected_hiddens = project_onto_direction(
            representation_pairs[layer].to("cpu").numpy(), directions[layer]
        )

        # order is [positive, negative, positive, negative, ...]
        positive_smaller_mean = np.mean(
            [
                projected_hiddens[i] < projected_hiddens[i + 1]
                for i in range(0, representation_pairs.shape[1], 2)
            ]
        )
        positive_larger_mean = np.mean(
            [
                projected_hiddens[i] > projected_hiddens[i + 1]
                for i in range(0, representation_pairs.shape[1], 2)
            ]
        )

        if positive_smaller_mean > positive_larger_mean:  # type: ignore
            directions[layer] *= -1

    return directions

In [ ]:
# target_path = "/home/sake/MusicGenRepEng_2.5s_ditto_hiddens_batch128_from1.5mean01s"
# opposite_path = "/home/sake/MusicGenRepEng_125tokens_uncond_hiddens_batch128_from1.5mean01s"

target_path = "/home/sake/MusicGenRepEng_2.5s_ditto_hiddens_batch128_meanssss"
opposite_path = "/home/sake/MusicGenRepEng_125tokens_uncond_hiddens_batch128_meanssss"

ditto_directions = get_directions(target_path, opposite_path)

### Get Control Vector - Text Pair

In [ ]:
import torch
from tqdm import tqdm
from pathlib import Path
import numpy as np
from sklearn.decomposition import PCA

In [ ]:
def project_onto_direction(H, direction):
    """Project matrix H (n, d_1) onto direction vector (d_2,)"""
    mag = np.linalg.norm(direction)
    assert not np.isinf(mag)
    return (H @ direction) / mag

In [ ]:
def get_directions(path):
    representation_pairs = []
    for p in tqdm(Path(path).rglob('*.pt')):
        loaded = torch.load(str(p), map_location=torch.device('cpu'))[-1]
        representation_pairs.append(loaded)

    representation_pairs = torch.cat(representation_pairs, dim=0)
    representation_pairs = representation_pairs.permute(1,0,2)

    relative_layer_hiddens = {}
    for layer, pair in enumerate(representation_pairs):
        relative_layer_hiddens[layer] = (
            pair[::2] - pair[1::2]
        )

    directions = {}
    for layer in range(len(relative_layer_hiddens)):
        # assert representation_pairs[layer].shape[0] == 110 * 2

        # fit layer directions
        train = np.vstack(
            relative_layer_hiddens[layer].to("cpu").numpy()
            - relative_layer_hiddens[layer].to("cpu").numpy().mean(axis=0, keepdims=True)
        )
        pca_model = PCA(n_components=1, whiten=False).fit(train)
        # shape (n_features,)
        directions[layer] = pca_model.components_.astype(np.float32).squeeze(axis=0)

        # calculate sign
        projected_hiddens = project_onto_direction(
            representation_pairs[layer].to("cpu").numpy(), directions[layer]
        )

        # order is [positive, negative, positive, negative, ...]
        positive_smaller_mean = np.mean(
            [
                projected_hiddens[i] < projected_hiddens[i + 1]
                for i in range(0, representation_pairs.shape[1], 2)
            ]
        )
        positive_larger_mean = np.mean(
            [
                projected_hiddens[i] > projected_hiddens[i + 1]
                for i in range(0, representation_pairs.shape[1], 2)
            ]
        )

        if positive_smaller_mean > positive_larger_mean:  # type: ignore
            directions[layer] *= -1

    return directions

In [ ]:
# clear_directions = get_directions('/home/sake/MusicGenRepEng_Dataset_50ms_clear_poor_mediummodel_norm_nob4layer')
energy_directions_b4 = get_directions('/home/sake/MusicGenRepEng_Dataset_50ms_energy_mediummodel_unnorm_b4layer')
# energy_directions_nob4 = get_directions('/home/sake/MusicGenRepEng_Dataset_50ms_energy_mediummodel_unnorm_nob4layer')
# happy_directions = get_directions('/home/sake/MusicGenRepEng_Dataset_50ms_happy_scary_mediummodel_norm_nob4layer')

In [ ]:
loaded = torch.load("/home/sake/MusicGenRepEng_Dataset_50ms_energy_mediummodel_unnorm_b4layer/electronica/Ambient/Ambient Techno/Orbital - Halcyon And On And On.pt")

In [ ]:
len(loaded)

In [ ]:
loaded[-1][:,0,:].shape

In [ ]:
a = torch.Tensor([1,2])
b = a.to(torch.float16)
c = b.numpy()
a.dtype, b.dtype, c.dtype

In [ ]:
energy_directions_b4[0].shape

### Get Control Vector- Text Pair [MISC]

In [ ]:
representation_pairs = []
for path in tqdm(Path('/home/sake/MusicGenRepEng_Dataset_50ms_clear_poor_mediummodel_norm_nob4layer').rglob('*.pt')):
    loaded = torch.load(str(path))[-1]
    representation_pairs.append(loaded)

In [ ]:
representation_pairs = torch.cat(representation_pairs, dim=0)
representation_pairs = representation_pairs.permute(1,0,2)
representation_pairs.shape

In [ ]:
relative_layer_hiddens = {}

In [ ]:
for layer, pair in enumerate(representation_pairs):
    relative_layer_hiddens[layer] = (
        pair[::2] - pair[1::2]
    )

In [ ]:
relative_layer_hiddens[0].shape

In [ ]:
# for i in range(48):
#     print(i, (relative_layer_hiddens[i][0]==0).all())

In [ ]:
directions = {}
for layer in range(len(relative_layer_hiddens)):
    # assert representation_pairs[layer].shape[0] == 110 * 2

    # fit layer directions
    train = np.vstack(
        relative_layer_hiddens[layer].to("cpu").numpy()
        - relative_layer_hiddens[layer].to("cpu").numpy().mean(axis=0, keepdims=True)
    )
    pca_model = PCA(n_components=1, whiten=False).fit(train)
    # shape (n_features,)
    directions[layer] = pca_model.components_.astype(np.float32).squeeze(axis=0)

    # calculate sign
    projected_hiddens = project_onto_direction(
        representation_pairs[layer].to("cpu").numpy(), directions[layer]
    )

    # order is [positive, negative, positive, negative, ...]
    positive_smaller_mean = np.mean(
        [
            projected_hiddens[i] < projected_hiddens[i + 1]
            for i in range(0, representation_pairs.shape[1], 2)
        ]
    )
    positive_larger_mean = np.mean(
        [
            projected_hiddens[i] > projected_hiddens[i + 1]
            for i in range(0, representation_pairs.shape[1], 2)
        ]
    )

    if positive_smaller_mean > positive_larger_mean:  # type: ignore
        directions[layer] *= -1

In [ ]:
len(directions)

In [ ]:
directions[47].shape

In [ ]:
energy_directions = directions

In [ ]:
happy_directions = directions

### Get Control Vector - A song pair

In [ ]:
ditto_reps = torch.load("/home/sake/Ditto-2-NewJeans_MusicGenRepEng_Dataset_hidden_states_every20s_10t.pt")

In [ ]:
dexter_reps = torch.load("/home/sake/Ricardo Villalobos - Dexter [SED008]_MusicGenRepEng_Dataset_hidden_states_every20s_10t.pt")

In [ ]:
ditto_reps[0][-1].shape

In [ ]:
target = torch.stack([rep[-1] for rep in ditto_reps], dim=0)
target = target.squeeze(1).permute(1, 0, 2).cpu()
target.shape

In [ ]:
reps = torch.stack([rep[-1] for rep in dexter_reps], dim=0)
reps = reps.squeeze(1).permute(1, 0, 2).cpu()
reps.shape

In [ ]:
reps = torch.stack([rep[-1] for rep in reps], dim=0)
reps = reps.squeeze(1).permute(1, 0, 2).cpu()
reps.shape

In [ ]:
import torch
from tqdm import tqdm
from pathlib import Path
import numpy as np
from sklearn.decomposition import PCA

In [ ]:
def project_onto_direction(H, direction):
    """Project matrix H (n, d_1) onto direction vector (d_2,)"""
    mag = np.linalg.norm(direction)
    assert not np.isinf(mag)
    return (H @ direction) / mag

In [ ]:
# Get Difference

diffs = target.cpu() - reps.cpu() # target - others (pos - neg)
diffs.shape

In [ ]:
# Avg or Last Hidden State

directions = {}
for layer in tqdm(range(diffs.shape[0])):
    # assert diff[layer].shape[0] == len(inputs) * 2

    # fit layer directions
    train = np.vstack(
        diffs[layer]
        # - diffs[layer].mean(axis=0, keepdims=True)
    )
    pca_model = PCA(n_components=1, whiten=False).fit(train)
    # shape (n_features,)
    directions[layer] = pca_model.components_.astype(np.float32).squeeze(axis=0)
    # print(directions[layer].shape)
    # calculate sign
    # projected_hiddens = project_onto_direction(
    #     reps[layer], directions[layer]
    # )
    # # print(projected_hiddens[0])
    # target_projected_hiddens = project_onto_direction(
    #     target[layer], directions[layer]
    # )
    # # print(target_projected_hiddens[0])

    # # order is [positive, negative, positive, negative, ...]
    # positive_smaller_mean = np.mean(
    #     [
    #         target_projected_hiddens[0] < projected_hiddens[i] # target is smaller than others
    #         for i in range(0, reps.shape[1])
    #     ]
    # )
    # positive_larger_mean = np.mean(
    #     [
    #         target_projected_hiddens[0] > projected_hiddens[i] # target is larger than others
    #         for i in range(0, reps.shape[1])
    #     ]
    # )

    # if positive_smaller_mean > positive_larger_mean:  # type: ignore
    #     directions[layer] *= -1

In [ ]:
ditto_smells_directions = directions

### Get Control Vector

In [ ]:
import torch
from tqdm import tqdm
from pathlib import Path
from sklearn.decomposition import PCA

In [ ]:
reps = []
for path in tqdm(Path('/home/sake/MusicGenRepEng_Dataset_hidden_states_30-60_mid10_non_avg_smallmodel').rglob('*.pt')):
    rep = torch.load(path)
    reps.append(rep.cpu())
reps = torch.stack(reps)
reps = reps.squeeze(1)
# reps = reps.permute(1, 0, 2) # (layers, batch, hidden_states)
reps = reps.permute(2, 0, 1, 3) # (layers, batch, timesteps, hidden_states)
# reps = reps[:,:,-1] # Last hidden_state
reps.shape

In [ ]:
target = torch.load("/home/sake/MusicGenRepEng_Dataset_hidden_states_30-60_mid10/Ditto-2-NewJeans.pt")
target = target.permute(1, 0, 2).cpu() # (layers, batch, hidden_states)
target.shape

In [ ]:
target = rep_vec.cpu()
target = target.permute(2, 0, 1, 3) # (layers, batch, timesteps, hidden_states)
# target = target[:,:,-1] # Last hidden_state
target.shape

In [ ]:
# Get Difference

diffs = target.cpu() - reps.cpu() # target - others (pos - neg)
diffs.shape

In [ ]:
import numpy as np

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
def project_onto_direction(H, direction):
    """Project matrix H (n, d_1) onto direction vector (d_2,)"""
    mag = np.linalg.norm(direction)
    assert not np.isinf(mag)
    return (H @ direction) / mag

In [ ]:
reps[0].shape

In [ ]:
# Avg or Last Hidden State

directions = {}
for layer in tqdm(range(diffs.shape[0])):
    # assert diff[layer].shape[0] == len(inputs) * 2

    # fit layer directions
    train = np.vstack(
        diffs[layer]
        - diffs[layer].mean(axis=0, keepdims=True)
    )
    pca_model = PCA(n_components=1, whiten=False).fit(train)
    # shape (n_features,)
    directions[layer] = pca_model.components_.astype(np.float32).squeeze(axis=0)
    # print(directions[layer].shape)
    # calculate sign
    projected_hiddens = project_onto_direction(
        reps[layer], directions[layer]
    )
    # print(projected_hiddens[0])
    target_projected_hiddens = project_onto_direction(
        target[layer], directions[layer]
    )
    # print(target_projected_hiddens[0])

    # order is [positive, negative, positive, negative, ...]
    positive_smaller_mean = np.mean(
        [
            target_projected_hiddens[0] < projected_hiddens[i] # target is smaller than others
            for i in range(0, reps.shape[1])
        ]
    )
    positive_larger_mean = np.mean(
        [
            target_projected_hiddens[0] > projected_hiddens[i] # target is larger than others
            for i in range(0, reps.shape[1])
        ]
    )

    if positive_smaller_mean > positive_larger_mean:  # type: ignore
        directions[layer] *= -1

In [ ]:
f_diffs = diffs.flatten(1,2).cpu()
f_target = target.flatten(1,2).cpu()
f_reps = reps.flatten(1,2).cpu()
f_diffs.shape, f_target.shape, f_reps.shape

In [ ]:
# Multiple Hidden States


directions = {}
for layer in tqdm(range(f_diffs.shape[0])):
    # assert diff[layer].shape[0] == len(inputs) * 2

    # fit layer directions
    train = np.vstack(
        f_diffs[layer]
        - f_diffs[layer].mean(axis=0, keepdims=True)
    )
    pca_model = PCA(n_components=1, whiten=False).fit(train)
    # shape (n_features,)
    directions[layer] = pca_model.components_.astype(np.float32).squeeze(axis=0)
    # calculate sign
    projected_hiddens = project_onto_direction(
        f_reps[layer], directions[layer]
    )
    target_projected_hiddens = project_onto_direction(
        f_target[layer], directions[layer]
    )

    # order is [positive, negative, positive, negative, ...]
    positive_smaller_mean = np.mean(
        [
            target_projected_hiddens[i%f_target.shape[1]] < projected_hiddens[i] # target is smaller than others
            for i in range(0, f_reps.shape[1])
        ]
    )
    positive_larger_mean = np.mean(
        [
            target_projected_hiddens[i%f_target.shape[1]] > projected_hiddens[i] # target is larger than others
            for i in range(0, f_reps.shape[1])
        ]
    )

    if positive_smaller_mean > positive_larger_mean:  # type: ignore
        directions[layer] *= -1

In [ ]:
directions[0].shape

In [ ]:
directions

In [ ]:
directions[0].shape

In [ ]:
torch.save(directions, "/home/sake/Ditto-2-NewJeans_MusicGenRepEng_Dataset_hidden_states_30-60_non_avg_smallmodel_directions.pth")

### Inference with Control Vector

In [ ]:
from audiocraft.utils.notebook import display_audio

In [ ]:
import torchaudio

music, sr = torchaudio.load('/home/sake/ditto_vocalless.mp3')

In [ ]:
import os
import random
import torch
import numpy as np
# From https://gist.github.com/gatheluck/c57e2a40e3122028ceaecc3cb0d152ac
def set_all_seeds(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [ ]:
model.set_generation_params(
    use_sampling=True,
    top_k=250,
    duration=20,
    two_step_cfg=False
)

In [ ]:
n = 4
set_all_seeds(41)
res = model.generate_continuation(
    music[:,int(sr*30.00):int(sr*30.1)].repeat(n,1,1), sr, ['']*n, 
    progress=True)
display_audio(res, 32000)

In [ ]:
n = 4
set_all_seeds(42)
res = model.generate_continuation_with_control_vectors(music[:,int(sr*30.00):int(sr*30.10)].repeat(n,1,1), sr, 
                                                       control_vectors=[ditto_directions, ditto_dist_mean], coefficients=[1], sustains=[1000], ramps=[250],
                                                       before_layer=False, descriptions=[""]*n, progress=True)
display_audio(res, 32000)

In [ ]:
n = 8
set_all_seeds(42)
res = model.generate_with_control_vectors(descriptions=[""]*n, control_vectors=[ditto_directions, ditto_dist_mean], coefficients=[+1], sustains=[0], ramps=[50],
                                                       before_layer=False, progress=True)
display_audio(res, 32000)

In [ ]:
n = 4
set_all_seeds(42)
res = model.generate_with_control_vectors(descriptions=[""]*n, control_vectors=[ditto_directions, ditto_dist_mean], coefficients=[1], sustains=[1000], ramps=[10],
                                                       before_layer=False, progress=True)
display_audio(res, 32000)

In [ ]:
n = 4
set_all_seeds(42)
res = model.generate_with_control_vectors(descriptions=[""]*n, control_vectors=[ditto_directions, ditto_dist_mean], coefficients=[2], sustains=[0], ramps=[10],
                                                       before_layer=False, progress=True)
display_audio(res, 32000)

In [ ]:
n = 4
set_all_seeds(42)
res = model.generate_continuation(
    music[:,:int(sr*5.00)].repeat(n,1,1), sr, ['rock, extremely sleepy']*n, 
    progress=True)
display_audio(res, 32000)

In [ ]:

n = 4
set_all_seeds(42)
res = model.generate_continuation_with_control_vectors(music[:,:int(sr*5.00)].repeat(n,1,1), sr, 
                                                       control_vectors=[energy_directions], coefficients=[0.3], sustains=[1000], ramps=[250],
                                                       before_layer=False, descriptions=["pop"]*n, progress=True)
display_audio(res, 32000)

In [ ]:

n = 4
set_all_seeds(42)
res = model.generate_continuation_with_control_vectors(music[:,:int(sr*5.00)].repeat(n,1,1), sr, 
                                                       control_vectors=[energy_directions], coefficients=[-0.3], sustains=[1000], ramps=[250],
                                                       before_layer=False, descriptions=["pop"]*n, progress=True)
display_audio(res, 32000)

In [ ]:

n = 4
set_all_seeds(42)
res = model.generate_continuation_with_control_vectors(music[:,:int(sr*5.00)].repeat(n,1,1), sr, 
                                                       control_vectors=[energy_directions], coefficients=[0.1], sustains=[1000], ramps=[250],
                                                       before_layer=False, descriptions=["pop"]*n, progress=True)
display_audio(res, 32000)

In [ ]:

n = 4
set_all_seeds(42)
res = model.generate_continuation_with_control_vectors(music[:,:int(sr*5.00)].repeat(n,1,1), sr, 
                                                       control_vectors=[energy_directions], coefficients=[0.2], sustains=[1000], ramps=[250],
                                                       before_layer=False, descriptions=["pop"]*n, progress=True)
display_audio(res, 32000)

In [ ]:

n = 4
set_all_seeds(42)
res = model.generate_continuation_with_control_vectors(music[:,:int(sr*5.00)].repeat(n,1,1), sr, 
                                                       control_vectors=[energy_directions], coefficients=[-0.1], sustains=[1000], ramps=[250],
                                                       before_layer=False, descriptions=["pop"]*n, progress=True)
display_audio(res, 32000)

In [ ]:

n = 4
set_all_seeds(42)
res = model.generate_continuation_with_control_vectors(music[:,:int(sr*5.00)].repeat(n,1,1), sr, 
                                                       control_vectors=[energy_directions], coefficients=[-0.2], sustains=[1000], ramps=[250],
                                                       before_layer=False, descriptions=["pop"]*n, progress=True)
display_audio(res, 32000)

In [ ]:

n = 4
set_all_seeds(42)
res = model.generate_continuation_with_control_vectors(music[:,:int(sr*5.00)].repeat(n,1,1), sr, 
                                                       control_vectors=[energy_directions], coefficients=[0], sustains=[1000], ramps=[250],
                                                       before_layer=False, descriptions=["pop"]*n, progress=True)
display_audio(res, 32000)

In [ ]:

n = 4
set_all_seeds(42)
res = model.generate_continuation_with_control_vectors(music[:,:int(sr*5.00)].repeat(n,1,1), sr, 
                                                       control_vectors=[energy_directions], coefficients=[0.15], sustains=[1000], ramps=[250],
                                                       before_layer=False, descriptions=[None]*n, progress=True)
display_audio(res, 32000)

In [ ]:

n = 4
set_all_seeds(42)
res = model.generate_continuation_with_control_vectors(music[:,:int(sr*5.00)].repeat(n,1,1), sr, 
                                                       control_vectors=[energy_directions], coefficients=[0.2], sustains=[1000], ramps=[250],
                                                       before_layer=False, descriptions=[None]*n, progress=True)
display_audio(res, 32000)

In [ ]:

n = 4
set_all_seeds(42)
res = model.generate_continuation_with_control_vectors(music[:,:int(sr*5.00)].repeat(n,1,1), sr, 
                                                       control_vectors=[energy_directions], coefficients=[0], sustains=[1000], ramps=[250],
                                                       before_layer=False, descriptions=[None]*n, progress=True)
display_audio(res, 32000)

In [ ]:

n = 4
set_all_seeds(42)
res = model.generate_continuation_with_control_vectors(music[:,:int(sr*5.00)].repeat(n,1,1), sr, 
                                                       control_vectors=[energy_directions], coefficients=[-0.15], sustains=[1000], ramps=[250],
                                                       before_layer=False, descriptions=[None]*n, progress=True)
display_audio(res, 32000)

In [ ]:

n = 4
set_all_seeds(42)
res = model.generate_continuation_with_control_vectors(music[:,:int(sr*5.00)].repeat(n,1,1), sr, 
                                                       control_vectors=[energy_directions], coefficients=[-0.2], sustains=[1000], ramps=[250],
                                                       before_layer=False, descriptions=[None]*n, progress=True)
display_audio(res, 32000)

In [ ]:
# cocoeff 25 + clear
n = 4
set_all_seeds(42)
res = model.generate_continuation_with_control_vectors(music[:,:int(sr*5.00)].repeat(n,1,1), sr, 
                                                       control_vectors=[energy_directions, clear_directions], coefficients=[0.1, 0.1], sustains=[1000]*2, ramps=[250]*2,
                                                       before_layer=False, descriptions=["jazz"]*n, progress=True)
display_audio(res, 32000)

In [ ]:
# cocoeff 25 + clear
n = 4
set_all_seeds(42)
res = model.generate_continuation_with_control_vectors(music[:,:int(sr*5.00)].repeat(n,1,1), sr, 
                                                       control_vectors=[energy_directions, clear_directions], coefficients=[-0.1, 0.1], sustains=[1000]*2, ramps=[250]*2,
                                                       before_layer=False, descriptions=["jazz"]*n, progress=True)
display_audio(res, 32000)

In [ ]:
# cocoeff 25 + clear
n = 4
set_all_seeds(42)
res = model.generate_continuation_with_control_vectors(music[:,:int(sr*5.00)].repeat(n,1,1), sr, 
                                                       control_vectors=[energy_directions, clear_directions], coefficients=[0.1, -0.1], sustains=[1000]*2, ramps=[250]*2,
                                                       before_layer=False, descriptions=["jazz"]*n, progress=True)
display_audio(res, 32000)

In [ ]:
# cocoeff 25 + clear
n = 4
set_all_seeds(42)
res = model.generate_continuation_with_control_vectors(music[:,:int(sr*5.00)].repeat(n,1,1), sr, 
                                                       control_vectors=[energy_directions, clear_directions], coefficients=[-0.1, -0.1], sustains=[1000]*2, ramps=[250]*2,
                                                       before_layer=False, descriptions=["jazz"]*n, progress=True)
display_audio(res, 32000)

In [ ]:
# cocoeff 25 cvjazz
n = 4
set_all_seeds(42)
res = model.generate_continuation_with_control_vectors(music[:,:int(sr*5.00)].repeat(n,1,1), sr, 
                                                       control_vectors=[energy_directions_jazz], coefficients=[0.2], sustains=[1000], ramps=[250],
                                                       before_layer=False, descriptions=["jazz"]*n, progress=True)
display_audio(res, 32000)

In [ ]:
# cocoeff 10 cvjazz
n = 4
set_all_seeds(42)
res = model.generate_continuation_with_control_vectors(music[:,:int(sr*5.00)].repeat(n,1,1), sr, 
                                                       control_vectors=[energy_directions_jazz], coefficients=[0.1], sustains=[1000], ramps=[250],
                                                       before_layer=False, descriptions=["jazz"]*n, progress=True)
display_audio(res, 32000)

In [ ]:
# cocoeff 10 cvjazz
n = 4
set_all_seeds(42)
res = model.generate_continuation_with_control_vectors(music[:,:int(sr*5.00)].repeat(n,1,1), sr, 
                                                       control_vectors=[energy_directions_jazz], coefficients=[-0.1], sustains=[1000], ramps=[250],
                                                       before_layer=False, descriptions=["jazz"]*n, progress=True)
display_audio(res, 32000)

In [ ]:
# cocoeff 100 cvjazz
n = 4
set_all_seeds(42)
res = model.generate_continuation_with_control_vectors(music[:,:int(sr*5.00)].repeat(n,1,1), sr, 
                                                       control_vectors=[energy_directions_jazz], coefficients=[0.1], sustains=[1000], ramps=[250],
                                                       before_layer=False, descriptions=["jazz"]*n, progress=True)
display_audio(res, 32000)

In [ ]:
# cocoeff 100 cvjazz
n = 4
set_all_seeds(42)
res = model.generate_continuation_with_control_vectors(music[:,:int(sr*5.00)].repeat(n,1,1), sr, 
                                                       control_vectors=[energy_directions_jazz], coefficients=[-0.1], sustains=[1000], ramps=[250],
                                                       before_layer=False, descriptions=["jazz"]*n, progress=True)
display_audio(res, 32000)

In [ ]:
# cocoeff 50 cvjazz
n = 4
set_all_seeds(42)
res = model.generate_continuation_with_control_vectors(music[:,:int(sr*5.00)].repeat(n,1,1), sr, 
                                                       control_vectors=[energy_directions_jazz], coefficients=[0.1], sustains=[1000], ramps=[250],
                                                       before_layer=False, descriptions=["jazz"]*n, progress=True)
display_audio(res, 32000)

In [ ]:
# cocoeff 50 cvjazz
n = 4
set_all_seeds(42)
res = model.generate_continuation_with_control_vectors(music[:,:int(sr*5.00)].repeat(n,1,1), sr, 
                                                       control_vectors=[energy_directions_jazz], coefficients=[-0.1], sustains=[1000], ramps=[250],
                                                       before_layer=False, descriptions=["jazz"]*n, progress=True)
display_audio(res, 32000)

In [ ]:
# cocoeff 25 cvjazz
n = 4
set_all_seeds(42)
res = model.generate_continuation_with_control_vectors(music[:,:int(sr*5.00)].repeat(n,1,1), sr, 
                                                       control_vectors=[energy_directions_jazz], coefficients=[-0.2], sustains=[1000], ramps=[250],
                                                       before_layer=False, descriptions=["jazz"]*n, progress=True)
display_audio(res, 32000)

In [ ]:
# cocoeff 25
n = 4
set_all_seeds(42)
res = model.generate_continuation_with_control_vectors(music[:,:int(sr*5.00)].repeat(n,1,1), sr, 
                                                       control_vectors=[energy_directions], coefficients=[0.2], sustains=[1000], ramps=[250],
                                                       before_layer=False, descriptions=["jazz"]*n, progress=True)
display_audio(res, 32000)

In [ ]:
# cocoeff 25
n = 4
set_all_seeds(42)
res = model.generate_continuation_with_control_vectors(music[:,:int(sr*5.00)].repeat(n,1,1), sr, 
                                                       control_vectors=[energy_directions], coefficients=[-0.2], sustains=[1000], ramps=[250],
                                                       before_layer=False, descriptions=["jazz"]*n, progress=True)
display_audio(res, 32000)

In [ ]:
# cocoeff 25
n = 4
set_all_seeds(42)
res = model.generate_continuation_with_control_vectors(music[:,:int(sr*5.00)].repeat(n,1,1), sr, 
                                                       control_vectors=[energy_directions], coefficients=[0.1], sustains=[1000], ramps=[250],
                                                       before_layer=False, descriptions=["jazz"]*n, progress=True)
display_audio(res, 32000)

In [ ]:
# cocoeff 25
n = 4
set_all_seeds(42)
res = model.generate_continuation_with_control_vectors(music[:,:int(sr*5.00)].repeat(n,1,1), sr, 
                                                       control_vectors=[energy_directions], coefficients=[-0.1], sustains=[1000], ramps=[250],
                                                       before_layer=False, descriptions=["jazz"]*n, progress=True)
display_audio(res, 32000)

In [ ]:
# cocoeff 50
n = 4
set_all_seeds(42)
res = model.generate_continuation_with_control_vectors(music[:,:int(sr*5.00)].repeat(n,1,1), sr, 
                                                       control_vectors=[energy_directions], coefficients=[0.1], sustains=[1000], ramps=[250],
                                                       before_layer=False, descriptions=["jazz"]*n, progress=True)
display_audio(res, 32000)

In [ ]:
# cocoeff 50
n = 4
set_all_seeds(42)
res = model.generate_continuation_with_control_vectors(music[:,:int(sr*5.00)].repeat(n,1,1), sr, 
                                                       control_vectors=[energy_directions], coefficients=[-0.1], sustains=[1000], ramps=[250],
                                                       before_layer=False, descriptions=["jazz"]*n, progress=True)
display_audio(res, 32000)

In [ ]:
# cocoeff 50
n = 4
set_all_seeds(42)
res = model.generate_continuation_with_control_vectors(music[:,:int(sr*5.00)].repeat(n,1,1), sr, 
                                                       control_vectors=[energy_directions], coefficients=[-0.05], sustains=[1000], ramps=[250],
                                                       before_layer=False, descriptions=["jazz"]*n, progress=True)
display_audio(res, 32000)

In [ ]:
# rate 50
n = 4
set_all_seeds(42)
res = model.generate_continuation_with_control_vectors(music[:,:int(sr*5.00)].repeat(n,1,1), sr, 
                                                       control_vectors=[energy_directions], coefficients=[0.1], sustains=[1000], ramps=[250],
                                                       before_layer=False, descriptions=["jazz"]*n, progress=True)
display_audio(res, 32000)

In [ ]:
# rate 50
n = 4
set_all_seeds(42)
res = model.generate_continuation_with_control_vectors(music[:,:int(sr*5.00)].repeat(n,1,1), sr, 
                                                       control_vectors=[energy_directions], coefficients=[-0.1], sustains=[1000], ramps=[250],
                                                       before_layer=False, descriptions=["jazz"]*n, progress=True)
display_audio(res, 32000)

In [ ]:
# ~ 12
n = 4
set_all_seeds(42)
res = model.generate_continuation_with_control_vectors(music[:,:int(sr*5.00)].repeat(n,1,1), sr, 
                                                       control_vectors=[energy_directions], coefficients=[-0.1], sustains=[500], ramps=[250],
                                                       before_layer=False, descriptions=["jazz"]*n, progress=True)
display_audio(res, 32000)

In [ ]:
# ~ 24
n = 4
set_all_seeds(42)
res = model.generate_continuation_with_control_vectors(music[:,:int(sr*5.00)].repeat(n,1,1), sr, 
                                                       control_vectors=[energy_directions], coefficients=[0.05], sustains=[500], ramps=[250],
                                                       before_layer=False, descriptions=["jazz"]*n, progress=True)
display_audio(res, 32000)

In [ ]:
# ~ 24
n = 4
set_all_seeds(42)
res = model.generate_continuation_with_control_vectors(music[:,:int(sr*5.00)].repeat(n,1,1), sr, 
                                                       control_vectors=[energy_directions], coefficients=[-0.05], sustains=[500], ramps=[250],
                                                       before_layer=False, descriptions=["jazz"]*n, progress=True)
display_audio(res, 32000)

In [ ]:
# 24~
n = 4
set_all_seeds(42)
res = model.generate_continuation_with_control_vectors(music[:,:int(sr*5.00)].repeat(n,1,1), sr, 
                                                       control_vectors=[energy_directions], coefficients=[0.05], sustains=[500], ramps=[250],
                                                       before_layer=False, descriptions=["jazz"]*n, progress=True)
display_audio(res, 32000)

In [ ]:
# 24~
n = 4
set_all_seeds(42)
res = model.generate_continuation_with_control_vectors(music[:,:int(sr*5.00)].repeat(n,1,1), sr, 
                                                       control_vectors=[energy_directions], coefficients=[-0.05], sustains=[500], ramps=[250],
                                                       before_layer=False, descriptions=["jazz"]*n, progress=True)
display_audio(res, 32000)

In [ ]:
# > 36
n = 4
set_all_seeds(42)
res = model.generate_continuation_with_control_vectors(music[:,:int(sr*5.00)].repeat(n,1,1), sr, 
                                                       control_vectors=[energy_directions], coefficients=[0.05], sustains=[500], ramps=[250],
                                                       before_layer=False, descriptions=["jazz"]*n, progress=True)
display_audio(res, 32000)

In [ ]:
# > 36
n = 4
set_all_seeds(42)
res = model.generate_continuation_with_control_vectors(music[:,:int(sr*5.00)].repeat(n,1,1), sr, 
                                                       control_vectors=[energy_directions], coefficients=[-0.05], sustains=[500], ramps=[250],
                                                       before_layer=False, descriptions=["jazz"]*n, progress=True)
display_audio(res, 32000)

In [ ]:
# 48 stairs
n = 4
set_all_seeds(42)
res = model.generate_continuation_with_control_vectors(music[:,:int(sr*5.00)].repeat(n,1,1), sr, 
                                                       control_vectors=[energy_directions], coefficients=[0.1], sustains=[500], ramps=[250],
                                                       before_layer=False, descriptions=["jazz"]*n, progress=True)
display_audio(res, 32000)

In [ ]:
# 48 stairs
n = 4
set_all_seeds(42)
res = model.generate_continuation_with_control_vectors(music[:,:int(sr*5.00)].repeat(n,1,1), sr, 
                                                       control_vectors=[energy_directions], coefficients=[-0.1], sustains=[500], ramps=[250],
                                                       before_layer=False, descriptions=["jazz"]*n, progress=True)
display_audio(res, 32000)

In [ ]:
# happy +0.05 energy +0.05
n = 4
set_all_seeds(42)
res = model.generate_continuation_with_control_vectors(music[:,:int(sr*5.00)].repeat(n,1,1), sr, 
                                                       control_vectors=[energy_directions], coefficients=[-0.05], sustains=[500], ramps=[250],
                                                       before_layer=False, descriptions=["rock, sleepy"]*n, progress=True)
display_audio(res, 32000)

In [ ]:
# happy +0.05 energy +0.05
n = 4
set_all_seeds(42)
res = model.generate_continuation_with_control_vectors(music[:,:int(sr*5.00)].repeat(n,1,1), sr, 
                                                       control_vectors=[energy_directions], coefficients=[-0.1], sustains=[500], ramps=[250],
                                                       before_layer=False, descriptions=["rock, sleepy"]*n, progress=True)
display_audio(res, 32000)

In [ ]:
# happy +0.05 energy +0.05
n = 4
set_all_seeds(42)
res = model.generate_continuation_with_control_vectors(music[:,:int(sr*5.00)].repeat(n,1,1), sr, 
                                                       control_vectors=[energy_directions], coefficients=[-0.1], sustains=[100], ramps=[250],
                                                       before_layer=False, descriptions=["rock, sleepy"]*n, progress=True)
display_audio(res, 32000)

In [ ]:
# happy +0.05 energy +0.05
n = 4
set_all_seeds(42)
res = model.generate_continuation_with_control_vectors(music[:,:int(sr*5.00)].repeat(n,1,1), sr, 
                                                       control_vectors=[energy_directions], coefficients=[-0.2], sustains=[100], ramps=[250],
                                                       before_layer=False, descriptions=["rock, sleepy"]*n, progress=True)
display_audio(res, 32000)

In [ ]:
# happy +0.05 energy +0.05
n = 4
set_all_seeds(42)
res = model.generate_continuation_with_control_vectors(music[:,:int(sr*5.00)].repeat(n,1,1), sr, 
                                                       control_vectors=[energy_directions], coefficients=[-0.05], sustains=[100], ramps=[250],
                                                       before_layer=False, descriptions=["rock"]*n, progress=True)
display_audio(res, 32000)

In [ ]:
# happy +0.05 energy +0.05
n = 4
set_all_seeds(42)
res = model.generate_continuation_with_control_vectors(music[:,:int(sr*5.00)].repeat(n,1,1), sr, 
                                                       control_vectors=[energy_directions], coefficients=[-0.05], sustains=[500], ramps=[250],
                                                       before_layer=False, descriptions=["rock"]*n, progress=True)
display_audio(res, 32000)

In [ ]:
# happy +0.05 energy +0.05
n = 4
set_all_seeds(42)
res = model.generate_continuation_with_control_vectors(music[:,:int(sr*5.00)].repeat(n,1,1), sr, 
                                                       control_vectors=[energy_directions], coefficients=[-0.2], sustains=[500], ramps=[500],
                                                       before_layer=False, descriptions=["rock"]*n, progress=True)
display_audio(res, 32000)

In [ ]:
# happy +0.05 energy +0.05
n = 4
set_all_seeds(42)
res = model.generate_continuation_with_control_vectors(music[:,:int(sr*5.00)].repeat(n,1,1), sr, 
                                                       control_vectors=[energy_directions], coefficients=[-0.1], sustains=[750], ramps=[500],
                                                       before_layer=False, descriptions=["rock"]*n, progress=True)
display_audio(res, 32000)

In [ ]:
# happy +0.05 energy +0.05
n = 4
set_all_seeds(42)
res = model.generate_continuation_with_control_vectors(music[:,:int(sr*5.00)].repeat(n,1,1), sr, 
                                                       control_vectors=[energy_directions], coefficients=[-0.15], sustains=[500], ramps=[250],
                                                       before_layer=False, descriptions=["rock"]*n, progress=True)
display_audio(res, 32000)

In [ ]:
# happy +0.05 energy +0.05
n = 4
set_all_seeds(42)
res = model.generate_continuation_with_control_vectors(music[:,:int(sr*5.00)].repeat(n,1,1), sr, 
                                                       control_vectors=[energy_directions], coefficients=[-0.12], sustains=[500], ramps=[250],
                                                       before_layer=False, descriptions=["rock"]*n, progress=True)
display_audio(res, 32000)

In [ ]:
# happy +0.05 energy +0.05
n = 4
set_all_seeds(42)
res = model.generate_continuation_with_control_vectors(music[:,:int(sr*5.00)].repeat(n,1,1), sr, 
                                                       control_vectors=[energy_directions], coefficients=[-0.07], sustains=[500], ramps=[250],
                                                       before_layer=False, descriptions=["rock"]*n, progress=True)
display_audio(res, 32000)

In [ ]:
# happy +0.05 energy +0.05
n = 4
set_all_seeds(42)
res = model.generate_continuation_with_control_vectors(music[:,:int(sr*5.00)].repeat(n,1,1), sr, 
                                                       control_vectors=[energy_directions], coefficients=[-0.2], sustains=[100], ramps=[250],
                                                       before_layer=False, descriptions=["rock"]*n, progress=True)
display_audio(res, 32000)

In [ ]:
# happy +0.05 energy +0.05
n = 4
set_all_seeds(42)
res = model.generate_continuation_with_control_vectors(music[:,:int(sr*5.00)].repeat(n,1,1), sr, 
                                                       control_vectors=[energy_directions], coefficients=[-0.3], sustains=[100], ramps=[250],
                                                       before_layer=False, descriptions=["rock"]*n, progress=True)
display_audio(res, 32000)

In [ ]:
# happy +0.05 energy +0.05
n = 4
set_all_seeds(42)
res = model.generate_continuation_with_control_vectors(music[:,:int(sr*5.00)].repeat(n,1,1), sr, 
                                                       control_vectors=[energy_directions], coefficients=[-0.3], sustains=[50], ramps=[150],
                                                       before_layer=False, descriptions=["rock"]*n, progress=True)
display_audio(res, 32000)

In [ ]:
# happy +0.05 energy +0.05
n = 4
set_all_seeds(42)
res = model.generate_continuation_with_control_vectors(music[:,:int(sr*5.00)].repeat(n,1,1), sr, 
                                                       control_vectors=[energy_directions], coefficients=[-0.5], sustains=[1], ramps=[10],
                                                       before_layer=False, descriptions=["rock"]*n, progress=True)
display_audio(res, 32000)

In [ ]:
# happy +0.05 energy +0.05
n = 4
set_all_seeds(42)
res = model.generate_continuation_with_control_vectors(music[:,:int(sr*5.00)].repeat(n,1,1), sr, 
                                                       control_vectors=[energy_directions], coefficients=[-0.05], sustains=[100], ramps=[250],
                                                       before_layer=False, descriptions=["pop"]*n, progress=True)
display_audio(res, 32000)

In [ ]:
# happy +0.05 energy +0.05
n = 4
set_all_seeds(42)
res = model.generate_continuation_with_control_vectors(music[:,:int(sr*0.02)].repeat(n,1,1), sr, 
                                                       control_vectors=[happy_directions, energy_directions], coefficients=[0.05, 0.5], sustains=[100, 100], ramps=[250, 250],
                                                       before_layer=False, descriptions=["pop"]*n, progress=True)
display_audio(res, 32000)

In [ ]:
# happy +0.05 energy -0.05
n = 4
set_all_seeds(42)
res = model.generate_continuation_with_control_vectors(music[:,:int(sr*0.02)].repeat(n,1,1), sr, 
                                                       control_vectors=[happy_directions, energy_directions], coefficients=[0.05, -0.05], sustains=[100, 100], ramps=[250, 250],
                                                       before_layer=False, descriptions=["pop"]*n, progress=True)
display_audio(res, 32000)

In [ ]:
# happy -0.05 energy +0.05
n = 4
set_all_seeds(42)
res = model.generate_continuation_with_control_vectors(music[:,:int(sr*0.02)].repeat(n,1,1), sr, 
                                                       control_vectors=[happy_directions, energy_directions], coefficients=[-0.05, +0.05], sustains=[100, 100], ramps=[250, 250],
                                                       before_layer=False, descriptions=["pop"]*n, progress=True)
display_audio(res, 32000)

In [ ]:
# happy -0.05 energy -0.05
n = 4
set_all_seeds(42)
res = model.generate_continuation_with_control_vectors(music[:,:int(sr*0.02)].repeat(n,1,1), sr, 
                                                       control_vectors=[happy_directions, energy_directions], coefficients=[-0.05, -0.05], sustains=[100, 100], ramps=[250, 250],
                                                       before_layer=False, descriptions=["pop"]*n, progress=True)
display_audio(res, 32000)

In [ ]:
set_all_seeds(42)
output = model.generate(
    descriptions=[
        'happy jazz'
    ]*4 + ['jazz, happy']*4,
    progress=True, return_tokens=True
)
display_audio(output[0], sample_rate=32000)

In [ ]:
set_all_seeds(42)
output = model.generate(
    descriptions=[
        'sad jazz'
    ]*4 + ['jazz, sad']*4,
    progress=True, return_tokens=True
)
display_audio(output[0], sample_rate=32000)

In [ ]:
set_all_seeds(42)
output = model.generate(
    descriptions=[
        'rock, clear sound'
    ]*4 + ['rock, poor sound']*4,
    progress=True, return_tokens=True
)
display_audio(output[0], sample_rate=32000)

In [ ]:
set_all_seeds(42)
output = model.generate(
    descriptions=[
        'rock, high fidelity'
    ]*4 ,
    progress=True, return_tokens=True
)
display_audio(output[0], sample_rate=32000)

### Music Continuation

In [ ]:
import math
import torchaudio
import torch
from audiocraft.utils.notebook import display_audio

def get_bip_bip(bip_duration=0.125, frequency=440,
                duration=0.5, sample_rate=32000, device="cuda"):
    """Generates a series of bip bip at the given frequency."""
    t = torch.arange(
        int(duration * sample_rate), device="cuda", dtype=torch.float) / sample_rate
    wav = torch.cos(2 * math.pi * 440 * t)[None]
    tp = (t % (2 * bip_duration)) / (2 * bip_duration)
    envelope = (tp >= 0.5).float()
    return wav * envelope

In [ ]:
# Here we use a synthetic signal to prompt both the tonality and the BPM
# of the generated audio.
res = model.generate_continuation(
    get_bip_bip(0.125).expand(2, -1, -1), 
    32000, ['Jazz jazz and only jazz', 
            'Heartful EDM with beautiful synths and chords'], 
    progress=True)
display_audio(res, 32000)

In [ ]:
# You can also use any audio from a file. Make sure to trim the file if it is too long!
prompt_waveform, prompt_sr = torchaudio.load("../assets/bach.mp3")
prompt_duration = 2
prompt_waveform = prompt_waveform[..., :int(prompt_duration * prompt_sr)]
output = model.generate_continuation(prompt_waveform, prompt_sample_rate=prompt_sr, progress=True, return_tokens=True)
display_audio(output[0], sample_rate=32000)
if USE_DIFFUSION_DECODER:
    out_diffusion = mbd.tokens_to_wav(output[1])
    display_audio(out_diffusion, sample_rate=32000)

### Text-conditional Generation

In [ ]:
from audiocraft.utils.notebook import display_audio

output = model.generate(
    descriptions=[
        #'80s pop track with bassy drums and synth',
        #'90s rock song with loud guitars and heavy drums',
        #'Progressive rock drum and bass solo',
        #'Punk Rock song with loud drum and power guitar',
        #'Bluesy guitar instrumental with soulful licks and a driving rhythm section',
        #'Jazz Funk song with slap bass and powerful saxophone',
        'drum and bass beat with intense percussions'
    ],
    progress=True, return_tokens=True
)
display_audio(output[0], sample_rate=32000)
if USE_DIFFUSION_DECODER:
    out_diffusion = mbd.tokens_to_wav(output[1])
    display_audio(out_diffusion, sample_rate=32000)

### Melody-conditional Generation

In [ ]:
import torchaudio
from audiocraft.utils.notebook import display_audio

model = MusicGen.get_pretrained('facebook/musicgen-melody')
model.set_generation_params(duration=8)

melody_waveform, sr = torchaudio.load("../assets/bach.mp3")
melody_waveform = melody_waveform.unsqueeze(0).repeat(2, 1, 1)
output = model.generate_with_chroma(
    descriptions=[
        '80s pop track with bassy drums and synth',
        '90s rock song with loud guitars and heavy drums',
    ],
    melody_wavs=melody_waveform,
    melody_sample_rate=sr,
    progress=True, return_tokens=True
)
display_audio(output[0], sample_rate=32000)
if USE_DIFFUSION_DECODER:
    out_diffusion = mbd.tokens_to_wav(output[1])
    display_audio(out_diffusion, sample_rate=32000)

## Inference Grid

In [ ]:
grid_name = "LARGE_ENERGY_AFTERLAYER_NORMED_EVERYLAYER"
n = 8
sustain = 1500
ramp = 0
for path in tqdm(Path('/home/sake/demo songs').rglob('*.mp3')):
    genres = path.name.split('-')[0].split('_').append(None)
    for genre in genres:
        for coeff in range(-6, 7, 1):
            set_all_seeds(42)
            res = model.generate_continuation_with_control_vectors(music[:,:int(sr*5)].repeat(n,1,1), sr, 
                                                                control_vectors=[energy_directions], coefficients=[coeff/20.0], sustains=[sustain], ramps=[ramp],
                                                                before_layer=False, descriptions=[genre]*n, progress=True)
            for out in res:
                if genre is None:
                    genre_t = "None"
                else:
                    genre_t = genre
                out_path = f"/home/sake/grid_inference_outputs/{grid_name}/sus_{sustain}_ramp_{ramp}/{path.name.split('-')[1]}/{genre_t}/{coeff}.mp3"
                Path(out_path).parent.mkdir(parents=True, exist_ok=True)
                torchaudio.save(out_path, out, 32000)

In [ ]:
for path in tqdm(Path('/home/sake/demo songs').rglob('*.mp3')):
    print(path.name.split('-')[1])

In [ ]:
for coeff in range(-3, 4, 1):
    print(coeff)